In [1]:
## Bring in needed mods
import pandas as pd, numpy as np, glob

In [2]:
## Set date of phenotype data to use
filedate = 'Oct152018'

In [3]:
## Give example path to permutated
perms_path = '../FILES/PERMUT/Tecan_qtlruns_%s_curated_baselined_median21_AUC_Xstrain_*C_*_permu.csv.gz'%(filedate)

In [ ]:
## set df save path
savepath = '../FILES/Tecan_qtlruns_%s_curated_baselined_median21_AUC_Xstrain_QTL_Thresholds.csv'%(filedate)

In [4]:
## Set index to columns in permutation pvalue dfs
tc = 10

In [5]:
## Write needed ftn for analysis
def pval_cutoff(p,alpha=0.05):
    return np.round(np.sort(p)[int((1-alpha)*len(p))],3)

In [6]:
## Glob togehter the permutation files 
perms = glob.glob(perms_path)

In [7]:
## How many files do we have
len(perms)

9

In [8]:
## What do the paths look like?
perms

['../FILES/PERMUT/Tecan_qtlruns_Oct152018_curated_baselined_median21_AUC_Xstrain_30C_0.075_permu.csv.gz',
 '../FILES/PERMUT/Tecan_qtlruns_Oct152018_curated_baselined_median21_AUC_Xstrain_30C_0.0_permu.csv.gz',
 '../FILES/PERMUT/Tecan_qtlruns_Oct152018_curated_baselined_median21_AUC_Xstrain_30C_0.125_permu.csv.gz',
 '../FILES/PERMUT/Tecan_qtlruns_Oct152018_curated_baselined_median21_AUC_Xstrain_30C_0.175_permu.csv.gz',
 '../FILES/PERMUT/Tecan_qtlruns_Oct152018_curated_baselined_median21_AUC_Xstrain_37C_0.075_permu.csv.gz',
 '../FILES/PERMUT/Tecan_qtlruns_Oct152018_curated_baselined_median21_AUC_Xstrain_37C_0.0_permu.csv.gz',
 '../FILES/PERMUT/Tecan_qtlruns_Oct152018_curated_baselined_median21_AUC_Xstrain_37C_0.125_permu.csv.gz',
 '../FILES/PERMUT/Tecan_qtlruns_Oct152018_curated_baselined_median21_AUC_Xstrain_37C_0.175_permu.csv.gz',
 '../FILES/PERMUT/Tecan_qtlruns_Oct152018_curated_baselined_median21_AUC_Xstrain_39C_0.0_permu.csv.gz']

In [9]:
## Gather the column info
co_index = [pd.read_csv(f,compression='gzip',
                                   index_col=0).columns.tolist(
)[tc].split('_p')[0].split('_')[-2:] for f in perms]

In [10]:
## Check our work
co_index

[['30C', '0.075'],
 ['30C', '0.0'],
 ['30C', '0.125'],
 ['30C', '0.175'],
 ['37C', '0.075'],
 ['37C', '0.0'],
 ['37C', '0.125'],
 ['37C', '0.175'],
 ['39C', '0.0']]

In [11]:
## Calculate cutoffs
cutoffs = [pval_cutoff(pd.read_csv(f,compression='gzip',
                                   index_col=0).max(
    axis=0).iloc[tc:].values) 
for f in perms]

In [12]:
## Append to experiment index and modify it. 
for i,c in enumerate(co_index):
    c[0] = int(c[0][:-1])
    c[1] = float(c[1])
    c.append(cutoffs[i])

In [13]:
## Check work
assert [len(a) == 3 for a in co_index]

In [14]:
## Make dataframe
cutoff_df = pd.DataFrame(co_index,columns=['temp','amphB','cutoff'])

In [15]:
## Sort by temp and drug con
cutoff_df.sort_values(by=['temp','amphB'],inplace=True)

In [16]:
## view dataframe
cutoff_df

,temp,amphB,cutoff
1,30,0.000,4.139
0,30,0.075,4.033
2,30,0.125,4.074
3,30,0.175,4.130
5,37,0.000,4.098
4,37,0.075,4.160
6,37,0.125,4.069
7,37,0.175,4.080
8,39,0.000,3.944


In [18]:
## Save dataframe
cutoff_df.to_csv(savepath)